In [10]:
import os
import sys
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



In [11]:
sys.path.append(os.path.abspath('../src'))


In [12]:
from feature_extraction import extract_features

CLASSES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

In [13]:
 # Extract features from augmented dataset


X_train = np.load("../models/features_train_resnet18.npy")
y_train = np.load("../models/labels_train_resnet18.npy")

X_test = np.load("../models/features_test_resnet18.npy")
y_test = np.load("../models/labels_test_resnet18.npy")


In [14]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:


SVMmodel = SVC(kernel='rbf', C=30, gamma='scale', probability=True)
SVMmodel.fit(X_train, y_train)
y_pred = SVMmodel.predict(X_test)
print("SVM Validation Accuracy:", accuracy_score(y_test, y_pred))


# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # SVM grid
# svc = SVC(probability=True)
# params_svc = {'C':[1,10,30,100], 'gamma':['scale','auto']}
# gs = GridSearchCV(svc, params_svc, cv=cv, scoring='accuracy', n_jobs=-1)
# gs.fit(X, y)
# print("Best SVM", gs.best_score_, gs.best_params_)
# y_pred = gs.predict(X)
# print("Train acc", accuracy_score(y, y_pred))
# print(classification_report(y, y_pred, target_names=CLASSES))

SVM Validation Accuracy: 0.8906666666666667


In [16]:


 
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(X_train, y_train)

# 5. Evaluate
y_pred = knn.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("KNN Accuracy:", acc)

KNN Accuracy: 0.8293333333333334


In [17]:

def find_best_threshold(model, X_val, y_val):
    scores = model.decision_function(X_val)
    max_scores = np.max(scores, axis=1)

    # Only look at correctly predicted samples
    y_pred = model.predict(X_val)
    correct_mask = (y_pred == y_val)
    correct_scores = max_scores[correct_mask]

    # Choose threshold as minimum correct score - margin
    threshold = np.min(correct_scores) - 0.05

    return threshold
